In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime
import pytz
import scipy.stats


import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

In [2]:
teamURL = 'https://statsapi.web.nhl.com/api/v1/teams'

teams = requests.get(teamURL).json()
teams = teams['teams']

team_names = pd.DataFrame()

for team in teams:
    abbreviation = team['abbreviation']
    name = team['name']
    id = team['id']

    temp = pd.DataFrame({'abbreviation': [abbreviation], 'name': [name], 'id': [id]})

    team_names = pd.concat([team_names, temp], ignore_index=True)
    
# ----------------------------------------------------------------------#    
    
teamURL = 'https://statsapi.web.nhl.com/api/v1/teams/'
appendix = '/stats'

team_stats = pd.DataFrame()

for index, row in team_names.iterrows():

  fetchURL = teamURL + str(row['id']) + appendix

  # print(fetchURL)

  team = requests.get(fetchURL).json()
  stats = team['stats'][0]['splits'][0]['stat']
  name = team['stats'][0]['splits'][0]['team']['name']

  temp = pd.DataFrame([stats])
  temp['name'] = name

  team_stats = pd.concat([team_stats, temp], ignore_index=True)


team_stats = pd.merge(team_stats, team_names, on='name')
team_stats.set_index('name', inplace=True)

# Schedule Scrape

In [6]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/schedule'

# Date of RANKINGS
start = '?startDate=2023-10-31'

end = '&endDate=2024-04-18'

skedURL = baseURL + start + end

sked = requests.get(skedURL).json()

In [7]:
days = sked['dates']

eastern = pytz.timezone('US/Eastern')

In [8]:
# See teams with most games

teams_data = {}
games_data = []

for day in days:
    games = day['games']

    for game in games:
        home_team = game['teams']['home']['team']['name']
        away_team = game['teams']['away']['team']['name']

        if home_team not in teams_data:
            teams_data[home_team] = {'played': 0, 'home': 0, 'away': 0}
        if away_team not in teams_data:
            teams_data[away_team] = {'played': 0, 'home': 0, 'away': 0}

        teams_data[home_team]['played'] += 1
        teams_data[away_team]['played'] += 1
        teams_data[home_team]['home'] += 1
        teams_data[away_team]['away'] += 1

        games_data.append(game['gamePk'])

# Create a DataFrame
teams_df = pd.DataFrame.from_dict(teams_data, orient='index')
teams_df = teams_df.sort_values(by=['played', 'home'], ascending=False)

teams_df

,played,home,away
St. Louis Blues,75,38,37
Dallas Stars,75,37,38
Colorado Avalanche,74,39,35
Toronto Maple Leafs,74,38,36
Vancouver Canucks,74,38,36
Arizona Coyotes,74,38,36
Florida Panthers,74,37,37
Edmonton Oilers,74,37,37
Los Angeles Kings,74,36,38
Nashville Predators,74,36,38


# Scrape Data for this season

In [9]:
baseURL = 'https://statsapi.web.nhl.com/api/v1/game/'
appendix = '/feed/live'

year = '202302'
game = 1
range_end = int(str(games_data[0]-1)[-4:])

date_format_game = "%d-%m-%Y"
date_format = "%d-%m-%Y"
date_format_birth = "%Y-%m-%d"

awaylogs = {}
homelogs = {}

# Subtract games not done on Sunday!
for i in range(range_end):
    
    gameURL = baseURL + year + str(i+1).zfill(4) + appendix
    
    print(gameURL)

    game = requests.get(gameURL).json()
    
    away = game['liveData']['boxscore']['teams']['away']['players']
    awayname = game['gameData']['teams']['away']['abbreviation']
    home = game['liveData']['boxscore']['teams']['home']['players']
    homename = game['gameData']['teams']['home']['abbreviation']
    birthday = game['gameData']['players']

    if game['liveData']['linescore']['currentPeriod'] > 3:
        OT = True
    else:
        OT = False

    awayteam = []

    date = game['gameData']['datetime']['dateTime']

    datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

    utc_timestamp = pytz.utc.localize(datetime_obj)
    eastern_timezone = pytz.timezone('US/Eastern')
    eastern_time = utc_timestamp.astimezone(eastern_timezone)

    formatted_date_string = eastern_time.strftime('%d-%m-%Y')
    game_date = datetime.strptime(formatted_date_string, date_format)

    for player in away:
        awayteam.append(player)

    for player in awayteam:
        name = away[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = away[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = away[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = away[player]['position']['abbreviation']
            stats['name'] = name

        awaylogs[player  + '-' + formatted_date_string] = stats
        awaylogs[player  + '-' + formatted_date_string]['position'] = position
        awaylogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        awaylogs[player  + '-' + formatted_date_string]['name'] = name
        awaylogs[player  + '-' + formatted_date_string]['team'] = awayname
        awaylogs[player  + '-' + formatted_date_string]['opponent'] = homename
        awaylogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        awaylogs[player  + '-' + formatted_date_string]['id'] = player
        awaylogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days


    hometeam = []

    # date = game['gameData']['datetime']['dateTime']

    for player in home:
        hometeam.append(player)

    for player in hometeam:
        name = home[player]['person']['fullName']
        try:
            birth_date = birthday[player]['birthDate']
            birthday_date = datetime.strptime(birth_date, date_format_birth)
            days_aged = game_date - birthday_date
        except:
            birth_date = '1900-01-01'
        stats = home[player]['stats']
        try: 
            if list(stats.keys())[0] == 'skaterStats':
                position = home[player]['position']['abbreviation']
                stats = stats['skaterStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                stats['evenTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['evenTimeOnIce'].split(':'))))
                stats['powerPlayTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['powerPlayTimeOnIce'].split(':'))))
                stats['shortHandedTimeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['shortHandedTimeOnIce'].split(':'))))
            if list(stats.keys())[0] == 'goalieStats':
                position = 'G'
                stats = stats['goalieStats']
    #             stats['name'] = name
                stats['position'] = position
                stats['timeOnIce'] = sum(int(x) * 60**i for i, x in enumerate(reversed(stats['timeOnIce'].split(':'))))
                if OT & (stats['decision'] == "L"):
                    stats['decision'] = 'OTL'
                shots_against = stats['shortHandedShotsAgainst'] + stats['evenShotsAgainst'] + stats['powerPlayShotsAgainst']
                if (stats['saves'] == shots_against) & ((stats['decision'] == "W") | (stats['decision'] == "OTL")):
                    stats['shutout'] = '1'
                else:
                    stats['shutout'] = '0'
        except:
            position = home[player]['position']['abbreviation']
            stats['name'] = name
            
                

        homelogs[player  + '-' + formatted_date_string] = stats
        homelogs[player  + '-' + formatted_date_string]['position'] = position
        homelogs[player  + '-' + formatted_date_string]['date'] = formatted_date_string
        homelogs[player  + '-' + formatted_date_string]['name'] = name
        homelogs[player  + '-' + formatted_date_string]['team'] = homename
        homelogs[player  + '-' + formatted_date_string]['opponent'] = awayname
        homelogs[player  + '-' + formatted_date_string]['birthdate'] = birth_date
        homelogs[player  + '-' + formatted_date_string]['id'] = player
        homelogs[player  + '-' + formatted_date_string]['aged'] = days_aged.days

https://statsapi.web.nhl.com/api/v1/game/2023020001/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020002/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020003/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020004/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020005/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020006/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020007/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020008/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020009/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020010/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020011/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020012/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020013/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020014/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020015/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020016/feed/live
https://

https://statsapi.web.nhl.com/api/v1/game/2023020135/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020136/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020137/feed/live
https://statsapi.web.nhl.com/api/v1/game/2023020138/feed/live


In [10]:
home_df = pd.DataFrame(homelogs)
home_df = home_df.transpose()

away_df = pd.DataFrame(awaylogs)
away_df = away_df.transpose()

columns_to_convert1 = ['timeOnIce', 'assists', 'goals', 'shots', 'hits', 'powerPlayGoals', 'powerPlayAssists', 'penaltyMinutes']
columns_to_convert2 = ['faceOffWins', 'faceoffTaken', 'takeaways', 'giveaways', 'shortHandedGoals', 'shortHandedAssists']
columns_to_convert3 = ['blocked', 'plusMinus', 'evenTimeOnIce', 'powerPlayTimeOnIce', 'shortHandedTimeOnIce', 'pim', 'shutout']
columns_to_convert4 = ['saves', 'powerPlaySaves', 'shortHandedSaves', 'evenSaves', 'shortHandedShotsAgainst', 'evenShotsAgainst', 'powerPlayShotsAgainst']

home_df[columns_to_convert1] = home_df[columns_to_convert1].fillna(0).astype(int)
home_df[columns_to_convert2] = home_df[columns_to_convert2].fillna(0).astype(int)
home_df[columns_to_convert3] = home_df[columns_to_convert3].fillna(0).astype(int)
home_df[columns_to_convert4] = home_df[columns_to_convert4].fillna(0).astype(int)

away_df[columns_to_convert1] = away_df[columns_to_convert1].fillna(0).astype(int)
away_df[columns_to_convert2] = away_df[columns_to_convert2].fillna(0).astype(int)
away_df[columns_to_convert3] = away_df[columns_to_convert3].fillna(0).astype(int)
away_df[columns_to_convert4] = away_df[columns_to_convert4].fillna(0).astype(int)

all_23_df = pd.concat([home_df, away_df], axis=0)

all_23_df['date'] = pd.to_datetime(all_23_df['date'], format='%d-%m-%Y')
all_23_df['birthdate'] = pd.to_datetime(all_23_df['birthdate'], format='%Y-%m-%d')
all_23_df = all_23_df.sort_values(by='date')

# Save point here for future import

In [11]:
all_23_df.to_csv('2023-24-to-date.csv', encoding='utf-8', index=False)

In [12]:
# 22-23 FANTASYPOINTS
for index, row in all_23_df.iterrows():
    d = 0
    so = 0
    w = 0

    if row['timeOnIce'] > 0:
        all_23_df.at[index, 'gamePlayed'] = 1


    if row['position'] == "G":
        if (row['shutout'] == 1) & (row['timeOnIce'] < 3446):
            all_23_df.at[index, 'shutout'] = 0
        if row['shutout'] == 1:
            so = 3
        else:
            so = 0
        if row['decision'] == "W":
            d = 4
            w += 1
        elif row['decision'] == "OTL":
            d = 1
        elif row['decision'] == "":
            d = 0
        else:
            d = 0
        shots_against = row['shortHandedShotsAgainst'] + row['evenShotsAgainst'] + row['powerPlayShotsAgainst']
        all_23_df.at[index, 'shots_against'] = shots_against
        ga = (shots_against - row['saves']) * -2
        all_23_df.at[index, 'goals_against'] = abs(ga)/2
        saves = row['saves'] * .2
        fp = d + so + ga + saves
        all_23_df.at[index, 'fantasyPoints'] = fp
        all_23_df.at[index, 'decisionPoints'] = d
        all_23_df.at[index, 'wins'] = w

    else:
        st = row['powerPlayGoals'] + row['powerPlayAssists'] + row['shortHandedGoals'] + row['shortHandedAssists']
        fp = (row['goals']*2) + row['assists'] + (st * .5) + (row['blocked'] * .5) + ((row['hits'] + row['shots']) * .1)
        all_23_df.at[index, 'fantasyPoints'] = fp
        all_23_df.at[index, 'specialTeams'] = st


In [13]:
# Remove all zero ice time games

all_23_df = all_23_df[all_23_df['timeOnIce'] != 0]

In [14]:
# Replace Sebastian Aho, D, with Sebastian Aho (D)

all_23_df.loc[(all_23_df['name'] == 'Sebastian Aho') & (all_23_df['position'] == 'D')] = all_23_df.loc[(all_23_df['name'] == 'Sebastian Aho') & (all_23_df['position'] == 'D')].replace('Sebastian Aho', 'Sebastian Aho (D)')

In [15]:
all_23_df

,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffPct,faceOffWins,...,shortHandedSavePercentage,evenStrengthSavePercentage,shutout,gamePlayed,fantasyPoints,specialTeams,shots_against,goals_against,decisionPoints,wins
ID8478519-10-10-2023,1002,0,0,1,0,0,0,0,60.0,9,...,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8477450-10-10-2023,774,1,1,3,2,0,0,0,40.0,6,...,NaN,NaN,0,1.0,5.0,0.0,NaN,NaN,NaN,NaN
ID8476473-10-10-2023,1078,0,0,0,1,0,0,0,NaN,0,...,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8475791-10-10-2023,1244,0,0,2,1,0,0,0,NaN,0,...,NaN,NaN,0,1.0,0.3,0.0,NaN,NaN,NaN,NaN
ID8480252-10-10-2023,555,0,1,2,0,0,0,0,54.55,6,...,NaN,NaN,0,1.0,2.2,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ID8476392-30-10-2023,1046,0,0,0,1,0,0,2,46.15,6,...,NaN,NaN,0,1.0,0.1,0.0,NaN,NaN,NaN,NaN
ID8476480-30-10-2023,952,0,0,1,0,0,0,0,40.0,4,...,NaN,NaN,0,1.0,0.6,0.0,NaN,NaN,NaN,NaN
ID8478398-30-10-2023,1396,0,0,5,0,0,0,0,NaN,0,...,NaN,NaN,0,1.0,0.5,0.0,NaN,NaN,NaN,NaN
ID8480878-30-10-2023,861,1,0,1,1,0,0,2,NaN,0,...,NaN,NaN,0,1.0,1.2,0.0,NaN,NaN,NaN,NaN


# Bring in previous seasons

In [16]:
last_season = pd.read_csv('2022-23.csv')
season_prior = pd.read_csv('2021-22.csv')
two_years_ago = pd.read_csv('2020-21.csv')


last_season['date'] = pd.to_datetime(last_season['date'])
season_prior['date'] = pd.to_datetime(season_prior['date'])
two_years_ago['date'] = pd.to_datetime(two_years_ago['date'])

last_season['birthdate'] = pd.to_datetime(last_season['birthdate'])
season_prior['birthdate'] = pd.to_datetime(season_prior['birthdate'])
two_years_ago['birthdate'] = pd.to_datetime(two_years_ago['birthdate'])

last_three = pd.concat([all_23_df, last_season, season_prior], axis=0)

last_three = last_three.loc[last_three['timeOnIce'] > 0]

In [17]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats = all_23_df.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats['gamePlayed'] = summary_stats['gamePlayed'].astype(int)
summary_stats['decisionPoints'] = summary_stats['decisionPoints'].astype(int)
summary_stats['wins'] = summary_stats['wins'].astype(int)
summary_stats['specialTeams'] = summary_stats['specialTeams'].astype(int)


summary_stats['FPP60'] = (summary_stats['fantasyPoints'] / summary_stats['timeOnIce'] * 3600).round(2)
summary_stats['FPPG'] = (summary_stats['fantasyPoints'] / summary_stats['gamePlayed']).round(2)

summary_stats = summary_stats.sort_values(by='FPPG', ascending=False)
# pd.set_option('display.max_rows', None)
summary_stats.set_index('name', inplace=True)

summary_stats.head()

,id,team,position,timeOnIce,gamePlayed,goals,assists,specialTeams,shots,hits,...,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decisionPoints,wins,shutout,fantasyPoints,FPP60,FPPG
name,,,,,,,,,,,,,,,,,,,,,
Jeremy Swayman,ID8480280,BOS,G,14330,4,0,0,0,117,0,...,80,7,84,26,16,4,1,31.4,7.89,7.85
Semyon Varlamov,ID8473575,NYI,G,7147,2,0,0,0,76,0,...,58,3,61,12,4,1,1,15.6,7.86,7.80
Anthony Stolarz,ID8476932,FLA,G,3569,1,0,0,0,28,0,...,23,2,23,3,4,1,0,7.4,7.46,7.40
Joseph Woll,ID8479361,TOR,G,13578,4,0,0,0,127,0,...,98,3,101,23,12,3,0,29.4,7.79,7.35
Jonathan Quick,ID8471734,NYR,G,8752,3,0,0,0,57,0,...,54,1,55,1,8,2,1,20.2,8.31,6.73


In [18]:
skaters_summary = summary_stats.loc[summary_stats['position'] != "G"]
skaters_summary = skaters_summary.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])

goalies_summary = summary_stats.loc[summary_stats['position'] == "G"]
goalies_summary = goalies_summary.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])

goalies = all_23_df.loc[all_23_df['position'] == 'G']

skaters = all_23_df.loc[all_23_df['position'] != 'G']
skaters['specialTeams'] = skaters['specialTeams'].astype(int)

In [19]:
# Add skaters remaining games

for index, row in skaters_summary.iterrows():
    team = row['team']
    gr = 82 - team_stats.loc[team_stats['abbreviation'] == team]['gamesPlayed']

    skaters_summary.loc[index, 'gamesRemain'] = int(gr)

    skaters_summary.loc[index, 'fantasyPace'] = row['FPPG'] * int(gr) + int(row['fantasyPoints'])


skaters_summary['gamesRemain'] = skaters_summary['gamesRemain'].astype(int)

In [20]:
# Add goaltenders TEAM TOI

for index, row in team_stats.iterrows():
    team_stats.at[index, 'creaseMins'] = goalies_summary.loc[goalies_summary['team'] == row['abbreviation']]['timeOnIce'].sum()

team_stats['creaseMins'] = team_stats['creaseMins'].astype(int)

In [21]:
# Add goaltenders crease share percentage

for index, row in goalies_summary.iterrows():
    team = row['team']
    gr = 82 - team_stats.loc[team_stats['abbreviation'] == team]['gamesPlayed']

    goalies_summary.at[index, 'gamesRemain'] = int(gr)
    cs = row['timeOnIce'] / team_stats.loc[team_stats['abbreviation'] == team]['creaseMins'].sum()


    goalies_summary.at[index, 'fantasyPace'] = (float(row['FPP60']) * int(gr) * cs) + int(row['fantasyPoints'])
    goalies_summary.at[index, 'creaseShare'] = cs * 100

goalies_summary['gamesRemain'] = goalies_summary['gamesRemain'].astype(int)
goalies_summary['fantasyPace'] = round(goalies_summary['fantasyPace'], 2)
goalies_summary['creaseShare'] = round(goalies_summary['creaseShare'], 2)

# Attempt injury Sutractions

In [22]:
url = "https://statsapi.web.nhl.com/api/v1/schedule?startDate=2023-10-30&endDate=2024-04-18"
response = requests.get(url)

# Get the JSON data
data = response.json()

# Create a list to store game information
game_info = []

# Iterate through the dates
for date_info in data['dates']:
        
    games = date_info['games']
    for game in games:
        
        date = game['gameDate']

        datetime_obj = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')

        utc_timestamp = pytz.utc.localize(datetime_obj)
        eastern_timezone = pytz.timezone('US/Eastern')
        eastern_time = utc_timestamp.astimezone(eastern_timezone)

        formatted_date_string = eastern_time.strftime('%d-%m-%Y')
        
        game_info.append({
            'Game ID': game['gamePk'],
            'Date': formatted_date_string,
            'Home Team': game['teams']['home']['team']['name'],
            'Home': team_names.loc[team_names['name'] == game['teams']['home']['team']['name']]['abbreviation'].iloc[0],
            'Away Team': game['teams']['away']['team']['name'],
            'Away': team_names.loc[team_names['name'] == game['teams']['away']['team']['name']]['abbreviation'].iloc[0]
        })

# Create a DataFrame
remain_schedule = pd.DataFrame(game_info)
remain_schedule['Date'] = pd.to_datetime(remain_schedule['Date'], format='%d-%m-%Y')

In [23]:
def missing_games (name, date, today):
    today = pd.to_datetime(today, format='%d-%m-%Y')

    ret_date = pd.to_datetime(date, format='%d-%m-%Y')

    team = all_23_df.loc[all_23_df['name'] == name]['team'][0]
    all_games = remain_schedule.loc[(remain_schedule['Home'] == team) | (remain_schedule['Away'] == team)]
    missed_games = all_games.loc[(all_games['Date'] > today) & (all_games['Date'] < date)]
    remain_games = skaters_summary.loc[skaters_summary.index == name]['gamesRemain'][0] - len(missed_games)
    return remain_games

In [24]:
# today = '30-10-2023'

# milanlucicstr = 'Milan Lucic'
# milanlucic = '18-11-2023'
# skaters_summary.loc[skaters_summary.index == 'Milan Lucic', 'gamesRemain'] = int(missing_games(milanlucicstr, milanlucic, today))

# taylorhallstr = 'Taylor Hall'
# taylorhall = '04-11-2023'
# skaters_summary.loc[skaters_summary.index == 'Taylor Hall', 'gamesRemain'] = int(missing_games(taylorhallstr, taylorhall, today))

# patriklainestr = 'Patrik Laine'
# patriklaine = '06-11-2023'
# skaters_summary.loc[skaters_summary.index == 'Patrik Laine', 'gamesRemain'] = int(missing_games(patriklainestr, patriklaine, today))

# robbyfabbristr = 'Robby Fabbri'
# robbyfabbri = '16-11-2023'
# skaters_summary.loc[skaters_summary.index == 'Robby Fabbri', 'gamesRemain'] = int(missing_games(robbyfabbristr, robbyfabbri, today))

# davidsavardstr = 'David Savard'
# davidsavard = '07-12-2023'
# skaters_summary.loc[skaters_summary.index == 'David Savard', 'gamesRemain'] = int(missing_games(davidsavardstr, davidsavard, today))

# thomaschabotstr = 'Thomas Chabot'
# thomaschabot = '01-12-2023'
# skaters_summary.loc[skaters_summary.index == 'Thomas Chabot', 'gamesRemain'] = int(missing_games(thomaschabotstr, thomaschabot, today))

# andreburakovskystr = 'Andre Burakovsky'
# andreburakovsky = '02-12-2023'
# skaters_summary.loc[skaters_summary.index == 'Andre Burakovsky', 'gamesRemain'] = int(missing_games(andreburakovskystr, andreburakovsky, today))

# gabrielvilardistr = 'Gabriel Vilardi'
# gabrielvilardi = '22-11-2023'
# skaters_summary.loc[skaters_summary.index == 'Gabriel Vilardi', 'gamesRemain'] = int(missing_games(gabrielvilardistr, gabrielvilardi, today))



In [25]:
skaters_summary.loc[skaters_summary.index == 'Taylor Hall']['gamesRemain'][0] 

73

# Summary Stats for rankings -- using last three to start

In [26]:
# Build summary stats for rankings

def first_non_na_position(x):
    non_na_values = x[x != 'N/A']
    return non_na_values.iloc[0] if not non_na_values.empty else 'N/A'

summary_stats_last_three = last_three.groupby('id').agg({
    'name': 'first',
    'team': 'first',
    'position': first_non_na_position,
    'timeOnIce': 'sum',
    'gamePlayed': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'specialTeams': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'evenTimeOnIce': 'sum',
    'powerPlayTimeOnIce': 'sum',
    'shortHandedTimeOnIce': 'sum',
    'saves': 'sum',
    'powerPlaySaves': 'sum',
    'shortHandedSaves': 'sum',
    'evenSaves': 'sum',
    'shortHandedShotsAgainst': 'sum',
    'evenShotsAgainst': 'sum',
    'powerPlayShotsAgainst': 'sum',
    'decisionPoints': 'sum',
    'wins': 'sum',
    # 'decision': 'sum',
    'shutout': 'sum',
    'fantasyPoints': 'sum'
}).reset_index()

summary_stats_last_three.set_index('name', inplace=True)


# summary_stats = summary_stats.rename(columns={'date': 'gp'})
# summary_stats['gp'] = summary_stats['gp'].apply(lambda x: len(x))
summary_stats_last_three['gamePlayed'] = summary_stats_last_three['gamePlayed'].astype(int)
summary_stats_last_three['decisionPoints'] = summary_stats_last_three['decisionPoints'].astype(int)
summary_stats_last_three['wins'] = summary_stats_last_three['wins'].astype(int)
summary_stats_last_three['specialTeams'] = summary_stats_last_three['specialTeams'].astype(int)


summary_stats_last_three['FPP60'] = (summary_stats_last_three['fantasyPoints'] / summary_stats_last_three['timeOnIce'] * 3600).round(2)
summary_stats_last_three['FPPG'] = (summary_stats_last_three['fantasyPoints'] / summary_stats_last_three['gamePlayed']).round(2)

summary_stats_last_three = summary_stats_last_three.sort_values(by='timeOnIce', ascending=False)
# pd.set_option('display.max_rows', None)

summary_stats_last_three = summary_stats_last_three[summary_stats_last_three['position'] != 'N/A']

summary_stats_last_three.info()

skaters_summary_last_three = summary_stats_last_three.loc[summary_stats_last_three['position'] != "G"]
skaters_summary_last_three = skaters_summary_last_three.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout'])

goalies_summary_last_three = summary_stats_last_three.loc[summary_stats_last_three['position'] == "G"]
goalies_summary_last_three = goalies_summary_last_three.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])


<class 'pandas.core.frame.DataFrame'>
Index: 1306 entries, Juuse Saros to Jett Alexander
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1306 non-null   object 
 1   team                     1306 non-null   object 
 2   position                 1306 non-null   object 
 3   timeOnIce                1306 non-null   int64  
 4   gamePlayed               1306 non-null   int32  
 5   goals                    1306 non-null   int64  
 6   assists                  1306 non-null   int64  
 7   specialTeams             1306 non-null   int32  
 8   shots                    1306 non-null   int64  
 9   hits                     1306 non-null   int64  
 10  blocked                  1306 non-null   int64  
 11  evenTimeOnIce            1306 non-null   int64  
 12  powerPlayTimeOnIce       1306 non-null   int64  
 13  shortHandedTimeOnIce     1306 non-null   int64  
 14  saves    

In [27]:
for index, row in skaters_summary.iterrows():
    last_three_fppg = skaters_summary_last_three.loc[skaters_summary_last_three['id'] == row['id']]['FPPG']
#     try:
    skaters_summary.at[index, 'last3FPPG'] = last_three_fppg[0]
#     except:
#         skaters_summary.at[index, 'last3FPPG'] = 0

for index, row in goalies_summary.iterrows():
    last_three_fpp60 = goalies_summary_last_three.loc[goalies_summary_last_three['id'] == row['id']]['FPP60']
    goalies_summary.at[index, 'last3FPPG'] = last_three_fpp60[0]



In [28]:
goalies_summary.to_csv('goalies.csv', encoding='utf-8')
skaters_summary.to_csv('skaters.csv', encoding='utf-8')

# Opponent Stats

In [29]:
opponent_stats = skaters.groupby('opponent').agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum'
    # Add more columns as needed
})

opponent_stats['FPAPG'] = round(opponent_stats['fantasyPoints'] / opponent_stats['date'], 2)
opponent_stats['FPAPG_Rank'] = opponent_stats['FPAPG'].rank(ascending=True, method='min').astype(int)

# opponent_stats['BPG'] = round(opponent_stats['blocked'] / opponent_stats['date'], 2)
# opponent_stats['BPG_Rank'] = opponent_stats['BPG'].rank(ascending=True, method='min').astype(int)

opponent_stats = opponent_stats.rename_axis('abbreviation')

opponent_stats = pd.merge(opponent_stats, team_names, on='abbreviation')

opponent_stats['powerPlayPoints'] = opponent_stats['powerPlayGoals'] + opponent_stats['powerPlayAssists']

opponent_stats = opponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

opponent_stats

,abbreviation,timeOnIce,assists,goals,shots,hits,blocked,fantasyPoints,date,FPAPG,FPAPG_Rank,name,id,powerPlayPoints
0,ANA,159033,53,27,289,127,122,227.6,9,25.29,15,Anaheim Ducks,24,36
1,ARI,141071,38,21,256,180,101,184.6,8,23.08,9,Arizona Coyotes,53,21
2,BOS,160305,25,14,265,178,139,168.3,9,18.70,1,Boston Bruins,6,3
3,BUF,159333,46,27,273,189,135,221.7,9,24.63,14,Buffalo Sabres,7,12
4,CAR,177308,60,37,261,153,188,285.9,10,28.59,29,Carolina Hurricanes,12,27
5,CBJ,160399,49,30,295,106,164,239.6,9,26.62,23,Columbus Blue Jackets,29,15
6,CGY,159296,55,33,263,137,145,239.0,9,26.56,22,Calgary Flames,20,9
7,CHI,158981,57,32,308,151,146,248.4,9,27.60,27,Chicago Blackhawks,16,12
8,COL,141925,29,20,230,125,133,174.0,8,21.75,4,Colorado Avalanche,21,6
9,DAL,126247,28,17,215,139,112,158.4,7,22.63,6,Dallas Stars,25,3


# Opponent Stats by Position

In [30]:
skaters_temp = skaters
# replace_dict = {'LW': 'W', 'RW': 'W'}
skaters_temp['position'] = skaters_temp['position'].replace('LW', 'W')
skaters_temp['position'] = skaters_temp['position'].replace('RW', 'W')
skaters_temp = skaters_temp.loc[skaters_temp['gamePlayed']>0]


opponent_stats_by_pos = skaters_temp.groupby(['opponent', 'position']).agg({
    'timeOnIce': 'sum',
    'assists': 'sum',
    'goals': 'sum',
    'shots': 'sum',
    'hits': 'sum',
    'blocked': 'sum',
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum',
    'team': 'unique'
    # Add more columns as needed
})
opponent_stats_by_pos

opponent_stats_by_pos['powerPlayPoints'] = opponent_stats_by_pos['powerPlayGoals'] + opponent_stats_by_pos['powerPlayAssists']

opponent_stats_by_pos = opponent_stats_by_pos.drop(columns=['powerPlayGoals', 'powerPlayAssists'])

opponent_stats_by_pos = opponent_stats_by_pos.rename(columns={'date': 'gamesPlayed'})
opponent_stats_by_pos

timeOnIce  assists  goals  shots  hits  blocked  \
opponent position                                                    
ANA      C             52769       20     10     99    41       27   
         D             61290       22      4     79    44       71   
         W             44974       11     13    111    42       24   
ARI      C             41914        9      7     94    57       18   
         D             55533       13      4     67    58       62   
...                      ...      ...    ...    ...   ...      ...   
WPG      D             63032       15      6     73    63       96   
         W             45673       13     11    102    55       29   
WSH      C             41217       17     13     84    38       22   
         D             55771       14      3     61    32       84   
         W             46413       16     10     98    41       24   

                   fantasyPoints  gamesPlayed  \
opponent position                               
ANA      C                  74.5            9   
         D                  83.8            9   
         W                  69.3            9   
ARI      C                  50.6            8   
         D                  66.5            8   
...                          ...          ...   
WPG      D                  92.1            9   
         W                  68.7            9   
WSH      C                  72.2            8   
         D                  73.3            8   
         W                  64.4            8   

                                                            team  \
opponent position                                                  
ANA      C              [VGK, CAR, DAL, ARI, BOS, CBJ, PHI, PIT]   
         D              [VGK, CAR, DAL, ARI, BOS, CBJ, PHI, PIT]   
         W              [VGK, CAR, DAL, ARI, BOS, CBJ, PHI, PIT]   
ARI      C                   [NJD, NYR, NYI, STL, ANA, LAK, CHI]   
         D                   [NJD, NYR, NYI, STL, ANA, LAK, CHI]   
...                                                          ...   
WPG      D         [CGY, FLA, LAK, VGK, EDM, STL, DET, MTL, NYR]   
         W         [CGY, FLA, LAK, VGK, EDM, STL, DET, MTL, NYR]   
WSH      C              [PIT, CGY, OTT, MTL, TOR, NJD, MIN, SJS]   
         D              [PIT, CGY, OTT, MTL, TOR, NJD, MIN, SJS]   
         W              [PIT, CGY, OTT, MTL, TOR, NJD, MIN, SJS]   

                   powerPlayPoints  
opponent position                   
ANA      C                      14  
         D                      12  
         W                      10  
ARI      C                       7  
         D                       4  
...                            ...  
WPG      D                       6  
         W                       7  
WSH      C                      12  
         D                       4  
         W                       5  

[96 rows x 10 columns]

# Goalie opponent stats

In [31]:
gopponent_stats = goalies.groupby('opponent').agg({
    'timeOnIce': 'sum',
    'goals_against': 'sum',
    'shots': 'sum',
#     'fantasyPoints': ['sum', ('mean', lambda x: round(x.mean(), 2))],
    'fantasyPoints': 'sum',
    'date': 'nunique',
    'powerPlayGoals': 'sum',
    'powerPlayAssists': 'sum',
#     'team': 'unique',
    'powerPlaySaves': 'sum',
    'powerPlayShotsAgainst': 'sum'
    # Add more columns as needed
})

gopponent_stats['FPAPG'] = round(gopponent_stats['fantasyPoints'] / gopponent_stats['date'], 2)
gopponent_stats['FPAPG_Rank'] = gopponent_stats['FPAPG'].rank(ascending=False, method='min').astype(int)

# gopponent_stats['SPG'] = round(gopponent_stats['shots'] / gopponent_stats['date'], 2)
# gopponent_stats['SPG_Rank'] = gopponent_stats['SPG'].rank(ascending=False, method='min').astype(int)

gopponent_stats = gopponent_stats.rename_axis('abbreviation')

gopponent_stats = pd.merge(gopponent_stats, team_names, on='abbreviation')

# gopponent_stats['powerPlayPoints'] = gopponent_stats['powerPlayGoals'] + gopponent_stats['powerPlayAssists']

gopponent_stats = gopponent_stats.drop(columns=['powerPlayGoals', 'powerPlayAssists', 'id'])

gopponent_stats


,abbreviation,timeOnIce,goals_against,shots,fantasyPoints,date,powerPlaySaves,powerPlayShotsAgainst,FPAPG,FPAPG_Rank,name
0,ANA,32367,27.0,250,8.6,9,37,41,0.96,22,Anaheim Ducks
1,ARI,28902,27.0,227,6.0,8,32,41,0.75,24,Arizona Coyotes
2,BOS,31781,26.0,277,3.2,9,48,53,0.36,25,Boston Bruins
3,BUF,32284,25.0,268,19.6,9,42,45,2.18,15,Buffalo Sabres
4,CAR,36469,36.0,351,12.0,10,54,64,1.20,19,Carolina Hurricanes
5,CBJ,32625,22.0,291,40.8,9,51,56,4.53,4,Columbus Blue Jackets
6,CGY,32481,18.0,280,47.4,9,43,48,5.27,2,Calgary Flames
7,CHI,32130,17.0,235,40.6,9,34,37,4.51,5,Chicago Blackhawks
8,COL,28802,23.0,274,19.2,8,44,50,2.40,13,Colorado Avalanche
9,DAL,25573,20.0,207,7.4,7,20,22,1.06,20,Dallas Stars


# Per-Minute Stats

In [36]:
single_games = all_23_df.sort_values('fantasyPoints', ascending=False)

In [45]:
goaltenders_single = single_games.loc[single_games['position'] == 'G']
skaters_single = single_games.loc[single_games['position'] != 'G']
defense_single = skaters_single.loc[skaters_single['position'] == 'D']
forwards_single = skaters_single.loc[skaters_single['position'] != 'D']

In [46]:
skaters_single = skaters_single.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout', 'decision', 'savePercentage', 'powerPlaySavePercentage', 'shortHandedSavePercentage', 'evenStrengthSavePercentage', 'shots_against', 'goals_against'])
defense_single = defense_single.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout', 'decision', 'savePercentage', 'powerPlaySavePercentage', 'shortHandedSavePercentage', 'evenStrengthSavePercentage', 'shots_against', 'goals_against'])
forwards_single = forwards_single.drop(columns=['saves', 'powerPlaySaves',	'shortHandedSaves',	'evenSaves',	'shortHandedShotsAgainst',	'evenShotsAgainst',	'powerPlayShotsAgainst',	'decisionPoints',	'wins',	'shutout', 'decision', 'savePercentage', 'powerPlaySavePercentage', 'shortHandedSavePercentage', 'evenStrengthSavePercentage', 'shots_against', 'goals_against'])

goaltenders_single = goaltenders_single.drop(columns=['goals',	'assists',	'specialTeams',	'hits',	'blocked',	'evenTimeOnIce',	'powerPlayTimeOnIce',	'shortHandedTimeOnIce'])


In [54]:
goaltenders_single.head(75)['opponent'].value_counts()

SJS    7
SEA    5
PIT    5
CGY    5
CHI    4
CBJ    4
EDM    4
WSH    4
TOR    3
ANA    3
FLA    3
COL    3
DET    2
WPG    2
LAK    2
NYR    2
OTT    2
ARI    2
BUF    2
NSH    2
STL    2
DAL    1
MIN    1
MTL    1
VAN    1
TBL    1
CAR    1
NYI    1
Name: opponent, dtype: int64

In [58]:
goaltenders_single.loc[goaltenders_single['shutout'] == 1]['opponent'].value_counts()

CBJ    2
COL    2
SJS    2
CHI    2
FLA    1
CAR    1
EDM    1
CGY    1
VAN    1
SEA    1
STL    1
WSH    1
ARI    1
Name: opponent, dtype: int64

In [138]:
import pymannkendall as mk


test_data = defense_single.loc[defense_single['name'] == 'Erik Gustafsson'].sort_values('date', ascending=True)
test_data = test_data['powerPlayTimeOnIce'].to_list()
result = mk.original_test(test_data)
print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
print(', '.join(formatted_times))

def convert_to_minutes_seconds(seconds):
    minutes = seconds // 60
    remaining_seconds = seconds % 60
    return f"{minutes}:{remaining_seconds:02d}"

Scott Perunovich's power-play ice time is no trend and the Tau score is 0.33
0:29, 0:50, 1:12, 1:08, 0:33, 2:02, 0:38, 3:21, 0:51


In [88]:
d_cull = defense_single['name'].value_counts()
d_cull = d_cull[d_cull == 1]
d_cull = d_cull.index.to_list()
d_list = defense_single['name'].unique()
d_list = d_list.tolist()
d_list = [name for name in d_list if name not in d_cull]

In [116]:

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('date', ascending=True)
    test_data = test_data['timeOnIce'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))

print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('date', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('date', ascending=True)
    test_data = test_data['powerPlayTimeOnIce'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('date', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))

Vince Dunn's ice time is increasing and the Tau score is 0.51
22:18, 22:34, 24:10, 23:26, 22:55, 26:01, 25:12, 24:28, 23:09, 26:45
Jaccob Slavin's ice time is increasing and the Tau score is 0.64
17:20, 20:53, 19:19, 17:53, 20:54, 18:15, 22:49, 23:54, 21:05, 24:21
Oliver Ekman-Larsson's ice time is decreasing and the Tau score is -0.79
26:32, 26:16, 25:19, 23:56, 22:04, 22:37, 23:53, 20:22
Jake Bean's ice time is decreasing and the Tau score is -0.61
18:06, 19:56, 19:40, 17:06, 15:58, 15:04, 15:21, 16:28, 15:20
Nick Perbix's ice time is increasing and the Tau score is 0.72
11:42, 15:35, 15:52, 12:27, 17:13, 16:09, 18:24, 18:51, 18:10
TJ Brodie's ice time is increasing and the Tau score is 0.86
18:04, 19:49, 18:52, 21:11, 20:46, 22:18, 22:23, 25:33
J.J. Moser's ice time is increasing and the Tau score is 0.71
14:22, 14:47, 14:38, 17:38, 16:25, 16:40, 20:01, 20:00
Nicolas Hague's ice time is decreasing and the Tau score is -0.64
21:25, 19:49, 22:46, 21:04, 21:10, 18:02, 19:30, 16:06, 14:

In [97]:
f_cull = forwards_single['name'].value_counts()
f_cull = f_cull[f_cull == 1]
f_cull = f_cull.index.to_list()
f_list = forwards_single['name'].unique()
f_list = f_list.tolist()
f_list = [name for name in f_list if name not in f_cull]

In [115]:

    
for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('date', ascending=True)
    test_data = test_data['timeOnIce'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))

print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('date', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num:.1f}' for num in test_data]
        print(', '.join(formatted_numbers))
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('date', ascending=True)
    test_data = test_data['powerPlayTimeOnIce'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('date', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))

Frank Vatrano's ice time is decreasing and the Tau score is -0.56
20:41, 20:50, 19:36, 15:38, 18:56, 20:00, 18:33, 16:53, 16:36
David Pastrnak's ice time is increasing and the Tau score is 0.83
17:24, 17:26, 18:09, 18:18, 18:46, 17:29, 20:10, 20:25, 21:17
Jack Roslovic's ice time is increasing and the Tau score is 0.86
9:31, 11:11, 12:33, 15:33, 19:31, 20:23, 19:51, 20:17
Chris Kreider's ice time is increasing and the Tau score is 0.61
14:28, 18:26, 18:29, 20:46, 16:10, 18:56, 18:31, 21:25, 21:05
Alex Iafallo's ice time is increasing and the Tau score is 0.78
14:28, 14:45, 16:29, 13:05, 19:23, 17:46, 19:54, 20:59, 21:50
Joel Eriksson Ek's ice time is increasing and the Tau score is 0.61
18:29, 16:41, 18:04, 19:18, 21:32, 21:43, 17:30, 22:34, 23:43
Dylan Strome's ice time is increasing and the Tau score is 0.71
14:53, 14:16, 16:28, 16:09, 17:38, 21:12, 21:55, 21:00
Brayden Point's ice time is decreasing and the Tau score is -0.56
22:13, 21:52, 19:34, 26:26, 19:16, 19:07, 18:00, 15:08, 2

In [136]:
hurricanes = all_23_df.loc[all_23_df['team'] == 'NYR'].sort_values('date')
hurricanes_pp = hurricanes.groupby('date').sum()

formatted_times = [convert_to_minutes_seconds(seconds) for seconds in hurricanes_pp['powerPlayTimeOnIce']]

print(', '.join(formatted_times))


24:35, 25:00, 23:35, 32:12, 10:00, 27:00, 22:30, 48:10, 22:35
